In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle

In [3]:
model=ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable=False
model=tf.keras.Sequential([model,GlobalMaxPooling2D()])
model.summary()

94765736/94765736 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


In [5]:
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result

In [6]:
filenames = []

for file in os.listdir('/kaggle/input/fashion-product-images-dataset/fashion-dataset/images'):
    filenames.append(os.path.join('/kaggle/input/fashion-product-images-dataset/fashion-dataset/images',file))
pickle.dump(filenames,open('filenames.pkl','wb'))

In [ ]:

feature_list = []

for file in tqdm(filenames):
    feature_list.append(extract_features(file,model))

pickle.dump(feature_list,open('embeddings.pkl','wb'))



In [4]:
from sklearn.neighbors import NearestNeighbors
neighbors=NearestNeighbors(n_neighbors=5,  algorithm="brute", metric='euclidean')

neighbors.fit(feature_list)
distances,indices=neighbors.kneighbors([normalized_result])
print(indices)

NameError: name 'feature_list' is not defined